In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
import torchvision
from torchvision import datasets, transforms

ImportError: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'OpenSSL 1.1.0i  14 Aug 2018'. See: https://github.com/urllib3/urllib3/issues/2168

In [2]:
# Checking GPU Availability and PyTorch Version
print(torch.cuda.is_available(), torch.__version__)

True 1.13.1+cu117


In [3]:
# Define the Autoencoder Model
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 16, 3, padding=1),  # 28*28*16
            nn.ReLU(),
            nn.MaxPool2d(2, 2),  # 14*14*16
            nn.Conv2d(16, 32, 3, padding=1),  # 14*14*32
            nn.ReLU(),
            nn.MaxPool2d(2, 2)  # 7*7*32
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(32, 16, 3, stride=2, padding=1),  # 14*14*16
            nn.ReLU(),
            nn.ConvTranspose2d(16, 1, 3, stride=2, padding=1),  # 28*28*1
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

NameError: name 'transforms' is not defined

In [ ]:
# Load MNIST Dataset
transform = transforms.Compose([transforms.ToTensor()])
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=256, shuffle=True)

# Visualize Sample Images
dataiter = iter(trainloader)
images, labels = dataiter.next()
plt.figure(figsize=(10, 2))
for i in range(10):
    plt.subplot(1, 10, i+1)
    plt.imshow(images[i].squeeze().numpy(), cmap='gray')
    plt.axis('off')
plt.show()


In [ ]:
# Define the Model, Loss Function, and Optimizer
model = Autoencoder()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:
# Training the Autoencoder
for epoch in range(10):
    running_loss = 0.0
    for data in trainloader:
        inputs, _ = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, inputs)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {running_loss}")



In [ ]:
# Visualize Predictions
dataiter = iter(trainloader)
images, _ = dataiter.next()
outputs = model(images)


In [ ]:
plt.figure(figsize=(10, 2))
for i in range(10):
    plt.subplot(1, 10, i+1)
    plt.imshow(outputs[i].detach().numpy().squeeze(), cmap='gray')
    plt.axis('off')
plt.show()